# GLOFRIM 2.0 - Interactive notebook

### Import libs

First of all, import all required libraries. We start with the ones actually needed to execute GLOFRIM.

In [ ]:
import netCDF4
from glofrim import Glofrim, PCR, CMF

And then other libs only needed for this notebook, e.g. for plotting

In [ ]:
%matplotlib inline
import netCDF4
from os.path import join
import numpy as np
import matplotlib.tri as tri
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib
from matplotlib import cm
import pandas as pd

We could also need some auxiliary functions for plotting etc

In [ ]:
def make_map(projection=ccrs.PlateCarree(), bounds=None, figsize=(6, 6)):
    fig, ax = plt.subplots(figsize=figsize, subplot_kw=dict(projection=projection))
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    if bounds:
        xmin, ymin, xmax, ymax = bounds
        ax.set_xlim([xmin, xmax])
        ax.set_ylim([ymin, ymax])
    return fig, ax

### Set paths to model files and executables

We now need to specify all paths to the model settings files, e.g. ini-file for PCR-GLOBWB or mdu-file for Delft3D FM.

In [ ]:
ini_file = r'glofrim_PCR2CMF_demo.ini'

In case you work with different computer environments on a shared directory, paths to the local model executables can be provided in a separate env-file.
NOTE: Don't confuse this with the environment.yml file!

### Initialize the run with 2-step initialization (step 1/2)

The model configuration files do not necessarily match with the GLOFRIM settings. They hence must be aligned and initialized first.

In [ ]:
cbmi = Glofrim()
cbmi.initialize_config(config_fn=ini_file)

In [ ]:
#- double checking which models are coupled
models = cbmi.bmimodels.keys()
print(models)

In [ ]:
#- we can also check which variables are exchanged between the models
cbmi.exchanges

### the grid module

In [ ]:
cbmi.bmimodels['CMF'].get_grid()
cbmi.bmimodels['PCR'].get_grid()

define coordinates of observatoin stations (here: Obidos)

In [ ]:
# update if you're not running the Amazon model
x_obs_coord = -55.081092
y_obs_coord = -2.172760

In [ ]:
obs_ind = dict()
obs_xy = dict()
for mod in models:
    obs_ind[mod] = cbmi.index(x_obs_coord, y_obs_coord, mod, in1d=False)
    obs_xy[mod] = cbmi.bmimodels[mod].grid.xy(ind=obs_ind[mod]) # not for ugrid
    
obs_ind

visualize the grid. use upastream area to color the 1d CMF network

In [ ]:
# read uparea for color
uparea_fn = join(cbmi.bmimodels['CMF']._mapdir, 'uparea.bin')
uparea = np.fromfile(uparea_fn, dtype=np.float32).reshape(cbmi.bmimodels['CMF'].grid.height, cbmi.bmimodels['CMF'].grid.width)
uparea = uparea.flat[cbmi.bmimodels['CMF'].grid._1d.us_idx]

In [ ]:
%matplotlib notebook
import matplotlib as mpl
from os.path import join

# plot
fig, ax = make_map(bounds=cbmi.bmimodels['PCR'].grid.bounds, figsize=(10,10))
grid_collection = cbmi.bmimodels['PCR'].grid.plot_2d(ax=ax)
# quiv_collection = cbmi.bmimodels['PCR'].grid._dd.plot_dd(ax=ax)
quiv_collection = cbmi.bmimodels['CMF'].grid.plot_1d(ax=ax, c=uparea, norm=mpl.colors.LogNorm(), cmap='Blues')
ax.scatter(x_obs_coord, y_obs_coord, color='r', s=36)

### Aligning simulation period of coupled model

GLOFRIM automatically sets the model time depending on the dates provided by the coupled models. That means that only the period will be modelled by GLOFRIM which both models have in common.

Let's first see whether GLOFRIM detects differences in start and end time of models and defines the right period for the coupled run:

In [ ]:
cbmi.set_attribute_value('CMF.MAP:CRIVHGT', './rivhgt.bin')

In [ ]:
for mod in models:
    start_time = cbmi.bmimodels[mod].get_start_time()
    end_time = cbmi.bmimodels[mod].get_end_time()
    print '{:s}: start date {}; end date {}'.format(mod, start_time.date(), end_time.date())

In [ ]:
start_time = cbmi.get_start_time()
end_time = cbmi.get_end_time()
print 'GLOFRIM: start date {}; end date {}'.format(start_time.date(), end_time.date())

We see that GLOFRIM picks the earlier end time from CMF and uses this as end date for the coupled run.

But maybe we just want to simulate a shorter period (e.g. for lunch lectures) and thus set it to a user-specified end date.

In [ ]:
# cbmi.set_end_time('2006-05-01')
new_end_time = cbmi.get_end_time()
end_time = new_end_time
print 'GLOFRIM: start date {}; end date {}'.format(start_time.date(), end_time.date())

### Initialize the run with 2-step initialization (step 2/2)

Great! We now have different models coupled in one coupled model instance and the configuration files are aligned. This instance, and therefore the individual models, can now be initialized!

In [ ]:
cbmi.initialize_model()

### Something easy for the start - basic skills of the BMI

Run one time step

In [ ]:
cbmi.update()

In [ ]:
PCR_runoff = cbmi.get_value('PCR.runoff')
PCR_runoff

### Run coupled model

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models.sources import ColumnDataSource
from bokeh.io import output_notebook, push_notebook
import time
from datetime import datetime, timedelta
%matplotlib inline
output_notebook()


discharge_vars = {'PCR': 'PCR.discharge', 'CMF': 'CMF.outflw'}
discharge_data = ColumnDataSource(data = {k: [] for k in ['time'] + discharge_vars.keys()})

p = figure(title="GLOFRIM discharge simulations", plot_height=350, plot_width=800, x_axis_type='datetime')
line_pcr = p.line("time", "PCR", source=discharge_data, color="red", legend='PCRGLOB-WB')
line_cmf = p.line("time", "CMF", source=discharge_data, color="green", legend='CaMa-Flood')
p.yaxis.bounds = (1e3, 5e5)
p.yaxis.axis_label = 'Discharge [m3/s]'
p.legend.location = 'top_left'
target = show(p, notebook_handle=True)

Then the coupled models are updated until the GLOFRIM end time is reached:

In [ ]:
while cbmi.get_current_time() < cbmi.get_end_time():
    cbmi.update()
    new_data = dict(time=[cbmi.get_current_time()])
    for mod in discharge_vars:
        new_data.update({mod: cbmi.get_value_at_indices(discharge_vars[mod], obs_ind[mod])})
    discharge_data.stream(new_data)
    push_notebook(handle=target)

To correctly close all model processes and variables, the coupled models are finalized

In [ ]:
cbmi.finalize()

Sweet, that's it!